In [6]:
import toponetx
from toponetx.transform.graph_to_simplicial_complex import graph_to_neighbor_complex
import networkx as nx

In [10]:
g = nx.Graph()
g.add_nodes_from([1,2,3,4,5,6])
g.add_edges_from([(1,2),(1,3),(1,4),(2,3),(2,4),(2,5),(3,4),(5,6)])
print(g)

Graph with 6 nodes and 8 edges


In [11]:
simplicial = graph_to_neighbor_complex(g)
for ss in simplicial.simplices:
    print(ss)

frozenset({1})
frozenset({2})
frozenset({3})
frozenset({4})
frozenset({5})
frozenset({6})
frozenset({1, 2})
frozenset({1, 3})
frozenset({1, 4})
frozenset({2, 3})
frozenset({2, 4})
frozenset({3, 4})
frozenset({1, 5})
frozenset({2, 5})
frozenset({3, 5})
frozenset({4, 5})
frozenset({2, 6})
frozenset({5, 6})
frozenset({1, 2, 3})
frozenset({1, 2, 4})
frozenset({1, 3, 4})
frozenset({2, 3, 4})
frozenset({1, 2, 5})
frozenset({1, 3, 5})
frozenset({1, 4, 5})
frozenset({2, 3, 5})
frozenset({2, 4, 5})
frozenset({3, 4, 5})
frozenset({2, 5, 6})
frozenset({1, 2, 3, 4})
frozenset({1, 2, 3, 5})
frozenset({1, 2, 4, 5})
frozenset({1, 3, 4, 5})
frozenset({2, 3, 4, 5})
frozenset({1, 2, 3, 4, 5})


In [145]:
print(simplicial.up_laplacian_matrix(0).toarray())

[[ 4. -1. -1. -1. -1.  0.]
 [-1.  5. -1. -1. -1. -1.]
 [-1. -1.  4. -1. -1.  0.]
 [-1. -1. -1.  4. -1.  0.]
 [-1. -1. -1. -1.  5. -1.]
 [ 0. -1.  0.  0. -1.  2.]]


In [20]:
import torch_geometric
import torch

In [21]:
from itertools import combinations

In [174]:
class CliqueComplexLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, complex_dim=2):
        super().__init__()
        self.complex_dim = complex_dim
        self.added_fields = []
        for i in range(1, complex_dim+1):
            self.added_fields += [f"incidence_{i}", f"laplacian_down_{i}", f"laplacian_up_{i}"]

    def forward(self, data: torch_geometric.data.Data) -> dict:
        results = {}
        n_nodes = data.x.shape[0]
        edge_index = torch_geometric.utils.to_undirected(data.edge_index)
        simplices = [set() for _ in range(self.complex_dim+1)]
        for n in range(n_nodes):
            neighbors, _, _, _ = torch_geometric.utils.k_hop_subgraph(
                n, 1, edge_index
            )
            if n not in neighbors:
                neighbors.append(n)
            neighbors = neighbors.numpy()
            neighbors = set(neighbors)
            for i in range(self.complex_dim+1):
                for c in combinations(neighbors, i+1):
                    simplices[i].add(tuple(c))
                    
        for i in range(self.complex_dim+1):
            simplices[i] = list(simplices[i])
            print(len(simplices[i]))
            print(simplices[i])
            print("-----------------------------")
        incidences = [torch.zeros(len(simplices[i]), len(simplices[i+1])) for i in range(self.complex_dim)]
        laplacians_up = [torch.zeros(len(simplices[i]), len(simplices[i])) for i in range(self.complex_dim)]
        laplacians_down = [torch.zeros(len(simplices[i+1]), len(simplices[i+1])) for i in range(self.complex_dim)]
        for i in range(self.complex_dim):
            for idx_i, s_i in enumerate(simplices[i]):
                for idx_i_1, s_i_1 in enumerate(simplices[i+1]):
                    if all(e in s_i_1 for e in s_i):
                        incidences[i][idx_i][idx_i_1] = 1
            degree = torch.diag(torch.sum(incidences[i],dim=1))
            laplacians_up[i] = 2*degree - torch.mm(incidences[i],torch.transpose(incidences[i],1,0))
            degree = torch.diag(torch.sum(incidences[i],dim=0))
            laplacians_down[i] = 2*degree - torch.mm(torch.transpose(incidences[i],1,0),incidences[i])
                        
        for i, field in enumerate(self.added_fields):
            if i%3==0:
                results[field] = incidences[int(i/3)]
            if i%3==1:
                results[field] = laplacians_up[int(i/3)]
            if i%3==2:
                results[field] = laplacians_down[int(i/3)]
        return results

In [175]:
data = torch_geometric.data.Data()
data.x = torch.zeros([6,1])
data.edge_index = torch.tensor([[0,0,0,1,1,1,2,4],[1,2,3,2,3,4,3,5]])

In [176]:
lift = CliqueComplexLifting()

results = lift(data)

6
[(2,), (5,), (4,), (1,), (0,), (3,)]
-----------------------------
12
[(0, 1), (2, 4), (1, 2), (0, 4), (3, 4), (1, 5), (0, 3), (1, 4), (2, 3), (0, 2), (4, 5), (1, 3)]
-----------------------------
11
[(0, 1, 4), (0, 2, 4), (0, 3, 4), (0, 1, 3), (1, 2, 3), (0, 2, 3), (1, 3, 4), (2, 3, 4), (0, 1, 2), (1, 4, 5), (1, 2, 4)]
-----------------------------


In [177]:
for key in lift.added_fields:
    print(key)
    print(results[key])

incidence_1
tensor([[0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]])
laplacian_down_1
tensor([[ 4.,  0., -1., -1., -1., -1.],
        [ 0.,  2., -1., -1.,  0.,  0.],
        [-1., -1.,  5., -1., -1., -1.],
        [-1., -1., -1.,  5., -1., -1.],
        [-1.,  0., -1., -1.,  4., -1.],
        [-1.,  0., -1., -1., -1.,  4.]])
laplacian_up_1
tensor([[ 2.,  0., -1., -1.,  0., -1., -1., -1.,  0., -1.,  0., -1.],
        [ 0.,  2., -1., -1., -1.,  0.,  0., -1., -1., -1., -1.,  0.],
        [-1., -1.,  2.,  0.,  0., -1.,  0., -1., -1., -1.,  0., -1.],
        [-1., -1.,  0.,  2., -1.,  0., -1., -1.,  0., -1., -1.,  0.],
        [ 0., -1.,  0., -1.,  2.,  0., -1., -1., -1.,  0., -1., -1.],
        [-1.,  0

In [178]:
import networkx as nx

In [182]:
class CellCyclesLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, aggregation="sum"):
        super().__init__()
        self.added_fields = []
        self.aggregation = aggregation
        self.added_fields = ["x1", 
                             "incidence_1", 
                             "laplacian_down_1", 
                             "laplacian_up_1",
                             "incidence_2", 
                             "laplacian_down_2", 
                             "laplacian_up_2"]

    def forward(self, data: torch_geometric.data.Data) -> dict:
        results = {}
        n_nodes = data.x.shape[0]
        edge_index = torch_geometric.utils.to_undirected(data.edge_index)
        neighborhood = torch.zeros((n_nodes,n_nodes))
        print(edge_index.to_dense())
        
        return results

In [183]:
lift2 = CellCyclesLifting()

results = lift2(data)

tensor([[0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5],
        [1, 2, 3, 0, 2, 3, 4, 0, 1, 3, 0, 1, 2, 1, 5, 4]])
